# Berechnung der Bogenlänge mit Python

__Manfred Brill, Hochschule Kaiserslautern__

In der Vorlesung hatten wir schon untersucht wie wir mit Hilfe der Approximation der Parameterkurve durch einen Polygonzug eine Näherung der Bogenlänge berechnen. Dabei haben wir auch festgehalten, dass diese berechneten Näherungen nie größer als die eigentliche Bogenlänge werden kann, und dass für immer feinere Polygonzüge die berechneten Ergebnisse dem gesuchten Wert immer näher kommen.

In diesem Notebook implementieren wir diese Berechnungsmethode und verwenden den Einheitskreis und die semikubischen Parabel als Anwendungsbeispiele. Für diese beiden Kurven können wir die Bogenlänge angeben.

Im Anschluss werfen wir noch einen kurzen Blick auf alternative Berechnungsmethoden.

## Vorbereitungen
Wir importieren Numpy, das überrascht nicht und implementieren unsere beiden Beispiele. Wie schon für die grafische Darstellung geben wir als Ergebnis der Funktionen den Polygonzug zurück.

In [17]:
import numpy as np


def circle(radius=1.0, a=0.0, b=1.0, n=256):
    t = np.linspace(a, b, n)
    x = radius*np.cos(2.0*np.pi*t)
    y = radius*np.sin(2.0*np.pi*t)
    return x, y


def semicubic(a=-1.0, b=1.0, n=256):
    t = np.linspace(a, b, n)
    return t*t, t*t*t

## Berechnung der Länge eines Polygonzugs
Die Formel für die Näherung der Bogenlänge enthält ein Summenzeichen. Ohne NumPy würden wir vermutlich eine for-Schleife implementieren, die sukzessive die Abstände der einzelnen Segmente berechnet und aufaddiert. Wie schon häufig gelingt es auch hier auf der Basis von NumPy diese Berechnung ohne explizite Schleife zu realisieren. Dabei verwenden wir auch Slicing um  auf den aktuellen Punkt und seinen Vorgänger zuzugreifen.

In [4]:
def polylineLength(x, y):
    return np.sum(np.sqrt((x[1:] - x[:-1])**2 + (y[1:] - y[:-1])**2))

## Beispiele
Jetzt können wir unsere Beispiele betrachten - wir berechnen Polygonzüge, berechnen eine Näherung und vergleichen dies mit den exakten Ergebnissen aus der Vorlesung.

Beim Einheitskreis wissen wir, dass der Umfang durch $2\pi$ gegeben ist:

In [12]:
correct = 2.0*np.pi
x, y = circle()
computed = polylineLength(x, y)
print("Berechnete Näherung:", computed)
print("Relativer Fehler: ", np.abs(computed-correct)/correct)

Berechnete Näherung: 6.283026362971604
Relativer Fehler:  2.5296756376146255e-05


Wir haben bereits für 256 Punkten im Polygonzug einen relativen Fehler, der kleiner als $10^{-4}$ ist.

Für die semikubische Parabel hatten wir mit Hilfe von Substitution für das Parameterintervall [-1, 1] von ung. 2.87942 erhalten. 

In [18]:
correct = 2.87942
x, y = semicubic()
computed = polylineLength(x, y)
print("Berechnete Näherung:", computed)
print("Relativer Fehler: ", np.abs(computed-correct)/correct)

Berechnete Näherung: 2.879384544978544
Relativer Fehler:  1.2313251090871323e-05


Bei der gleichen Anzahl von Punkten im Polygonzug wie für den Einheitskreis erhalten wir eine ähnliche Genauigkeit. Durch das Slicing ist die Berechnung sehr performant.

## Algernative Berechnungemethoden
Natürlich gibt es Alternativen zu der Verwendung der Polygonzüge. Die Bogenlänge ist als Integral über die Bahngeschwindigkeit der Parameterkurve definiert. Implementieren wir diese Bahngeschwindigkeit, dann können wir Quadraturformeln für die Näherung des Integrals einsetzen. Solche Quadraturformeln finden wir im Modul scipy.integrate.
Hier setzen wir die Funktion *simpson* ein, die die gleichnamige Quadraturformel realisiert. In SciPy finden wir weitere Funktionen für die Näherung von Integralen, aber wir werden uns auf die Simpson-Regel konzentrieren.

In [25]:
import scipy.integrate as integrate


def v(t):
    return np.sqrt(4.0*t*t + 9.0*t*t*t*t)


a = -1.0
b = 1.0
n = 256
x = np.linspace(a, b, n)
y = v(x)
computed = integrate.simpson(y, x)
relError = np.abs((computed - correct)/correct)
print('Berechnete Näherung =', computed)
print('Relativer Fehler: ', relError)

Berechnete Näherung = 2.879430230602587
Relativer Fehler:  3.5530081012388787e-06


Wir erhalten eine ähnliche Genauigkeit wie mit Polygonzügen, müssen dafür aber die Bahngeschwindigkeit aufstellen und implementieren.

Natürlich können wir auch das Modul SymPy einsetzen und das Integral mit symbolischer Rechnung lösen. Dazu laden wir SymPy und lassen dieses Modul auch die Ableitungen und den Integranden aufstellen. Die Genauigkeit die wir damit erhalten ist durchaus im Bereich dessen, was wir bisher auch erhalten haben. Aber wenn Sie dieses Notebook ausführen werden Sie bemerken, dass wir eine nicht unerhebliche Rechenzeit dafür erwarten müssen.

In [24]:
import sympy

t = sympy.symbols('t')
f = t**2
g = t**3
fprime = sympy.diff(f, t)
gprime = sympy.diff(g, t)
vel = sympy.sqrt(fprime**2 + gprime**2)
result = sympy.integrate(vel, (t, -1, 1))
computed = float(result)
relError = np.abs((computed - correct)/correct)
print('Näherung =', computed)
print('Relativer Fehler: ', relError)

Näherung = 2.87939453125
Relativer Fehler:  8.845097276566296e-06


## Benchmarks
Welche Berechnungsmethode wir einsetzen hängt sicher auch von den Randbedingungen ab. Führen wir einen Benchmark mit dem Modul *timeit* durch, dann erhalten wir bei 1000 Wiederholungen die folgenden Ergebnisse:

Methode | Mittlere Ausführungszeit in Mikrosekunden
--- | ---
Polygonzug | 29.496300000118936
Simpson-Quadratur | 101.60600000017439
SymPy | 4394854.636500002